<a href="https://colab.research.google.com/github/simonejiang7/EfficientAD/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download data

In [1]:
!pip install pytorch-lightning --quiet
!pip install omegaconf --quiet
!pip install anomalib --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.7/727.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.7/349.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.0/31.0 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.6/185.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 569.1/569.1 kB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 829.5/829.5 kB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import gdown
import os
from __future__ import annotations

import logging
import random
from enum import Enum

import torch
import torch.nn.functional as F
from torch import Tensor, nn
from torchvision import transforms


In [3]:
!gdown https://drive.google.com/uc?id=1iXlSy--RLuyrky1xEF89GD1cHkjUFuhg

Downloading...
From: https://drive.google.com/uc?id=1iXlSy--RLuyrky1xEF89GD1cHkjUFuhg
To: /content/mvtec_anomaly_detection.tar.xz
100% 5.26G/5.26G [00:59<00:00, 88.9MB/s]


In [6]:
!git clone gh repo clone simonejiang7/mvtec-anomaly-detection

/bin/bash: line 1: gh: command not found


In [ ]:
if not os.path.exists('/content/data'):
    os.mkdir('/content/data')
!tar -xf mvtec_anomaly_detection.tar.xz -C '/content/data'

In [ ]:
import logging
from anomalib.data.mvtec import MVTec
from anomalib.config import get_configurable_parameters
from anomalib.data import get_datamodule
from anomalib.data.utils import TestSplitMode
from anomalib.models import get_model
from anomalib.utils.callbacks import LoadModelCallback, get_callbacks
from anomalib.utils.loggers import configure_logger, get_experiment_logger
from pytorch_lightning import Trainer, seed_everything

In [ ]:
config = get_configurable_parameters(config_path = '/content/config.yaml')

/usr/local/lib/python3.10/dist-packages/anomalib/config/config.py:280: UserWarning: config.project.unique_dir is set to False. This does not ensure that your results will be written in an empty directory and you may overwrite files.
  warn(


In [ ]:
center_crop = config.dataset.get("center_crop")
if center_crop is not None:
    center_crop = (center_crop[0], center_crop[1])

datamodule = MVTec(
    root=config.dataset.path,
    category='bottle',
    image_size=(config.dataset.image_size[0], config.dataset.image_size[1]),
    center_crop=center_crop,
    normalization=config.dataset.normalization,
    train_batch_size=config.dataset.train_batch_size,
    eval_batch_size=config.dataset.eval_batch_size,
    num_workers=config.dataset.num_workers,
    task=config.dataset.task,
    transform_config_train=config.dataset.transform_config.train,
    transform_config_eval=config.dataset.transform_config.eval,
    test_split_mode=config.dataset.test_split_mode,
    test_split_ratio=config.dataset.test_split_ratio,
    val_split_mode=config.dataset.val_split_mode,
    val_split_ratio=config.dataset.val_split_ratio,
)

In [ ]:
model = get_model(config)
experiment_logger = get_experiment_logger(config)
callbacks = get_callbacks(config)
trainer = Trainer(**config.trainer, logger=experiment_logger, callbacks=callbacks)
logger.info("Training the model.")
trainer.fit(model=model, datamodule=datamodule)

/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/anomalib/utils/callbacks/__init__.py:142: UserWarning: Export option: None not found. Defaulting to no model export
  warnings.warn(f"Export option: {config.optimization.export_mode} not found. Defaulting to no model export")
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
INFO:py

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.
